This notebook was used in my experimentation of handing duplicate data before adding it to the database. There are essentially groups of transactions in the data with the same accession number that I believe were all filed together. These must be combined into a single database entry to reflect the entire transaction. 

In [33]:
import polars as pl 
import math

'''
function to handle the duplicate groups basically sums # shares and shares owned following trans, takes mean of price/share

** i messed up the database so before we can add the rest of the data to the database, we have to alter each db table to change the type of price/share from INTEGER to REAL this is shown 
in a different notebook in this folder. SHows exactly how it can be done, its pretty easy. **
'''
def process_group(group):
    first_row = group.head(1)  # Get the first row of each group
    return pl.DataFrame({
      "ACCESSION_NUMBER": first_row["ACCESSION_NUMBER"],
      "TRANS_SHARES": group["TRANS_SHARES"].sum(),  # Keep value from first row
      "TRANS_PRICEPERSHARE": round(group["TRANS_PRICEPERSHARE"].mean(),2), # round to 2 decimals 
      "SHRS_OWND_FOLWNG_TRANS": group["SHRS_OWND_FOLWNG_TRANS"].sum(),
  })

data_path = 'testData/NONDERIV_TRANS.tsv'

# columns to be extracted from TSV file
desired_columns = ['ACCESSION_NUMBER', 'TRANS_SHARES', 'TRANS_PRICEPERSHARE', 'TRANS_ACQUIRED_DISP_CD', 'SHRS_OWND_FOLWNG_TRANS']

# results in a lazy frame from using scan_csv()
lazyframe = pl.scan_csv(data_path, separator='\t')

# select only the desired columns after scanning in the csv 
extracted_frame = lazyframe.select(desired_columns).collect()
print(extracted_frame)

# Filter non-duplicate rows based on ACCESSION_NUMBER -> this will give us only the non duplicate rows which we will concat with the processed dups at the end
non_duplicates = extracted_frame.filter(pl.col('ACCESSION_NUMBER').is_duplicated() == False)

# empty df to add our newly processed rows to
empty_df = pl.DataFrame(schema=pl.Schema([
    ("ACCESSION_NUMBER", pl.Utf8),
    ("TRANS_SHARES", pl.Float64),
    ("TRANS_PRICEPERSHARE", pl.Float64),
    ("TRANS_ACQUIRED_DISP_CD", pl.Utf8),
    ("SHRS_OWND_FOLWNG_TRANS", pl.Float64),
]))

# group the rows with duplicates and filter it so it only contains rows that have duplicate ACCESSION Nums -> return a Groupby object which contains a tuple of a group key and a df of the group
# thus we need to account for the tuple in the loop and just pass in the dataframe. the group df is a completely separate df containing only the group of dups
group_frame = extracted_frame.filter(pl.col("ACCESSION_NUMBER").is_duplicated() == True).group_by(pl.col('ACCESSION_NUMBER'))

# dont care about the group_key for our purpose. just throw the group df into the process func
for group_key, group_df in group_frame:
    # add the dataframe that contains a single processed elem to the 'empty' dataframe 
    empty_df = pl.concat([empty_df, process_group(group_df)])

#concat the processed dups (empty_df) with the non dups to get final frame baby
final_df = pl.concat([empty_df, non_duplicates])

print(final_df)

shape: (29, 5)
┌────────────────────┬──────────────┬────────────────────┬────────────────────┬────────────────────┐
│ ACCESSION_NUMBER   ┆ TRANS_SHARES ┆ TRANS_PRICEPERSHAR ┆ TRANS_ACQUIRED_DIS ┆ SHRS_OWND_FOLWNG_T │
│ ---                ┆ ---          ┆ E                  ┆ P_CD               ┆ RANS               │
│ str                ┆ f64          ┆ ---                ┆ ---                ┆ ---                │
│                    ┆              ┆ f64                ┆ str                ┆ f64                │
╞════════════════════╪══════════════╪════════════════════╪════════════════════╪════════════════════╡
│ 0001250853-24-0000 ┆ 258010.0     ┆ 1.27               ┆ A                  ┆ 2.461832e6         │
│ 09                 ┆              ┆                    ┆                    ┆                    │
│ 0001250853-24-0000 ┆ 49990.0      ┆ 1.27               ┆ A                  ┆ 2.511822e6         │
│ 09                 ┆              ┆                    ┆                  